In [4]:
import numpy as np
import math
from gurobipy import *

In [5]:
# Initialize Data
'''
@N: set of locations
@T: # of days; t = 0, 1, 2, 3, 4, 5 
@D: units of cargo that need to be carried from location i to location j on day t
@C: empty repositioning cost from location i to location j
@H: daily holding cost for one unit of cargo
@N_Aircraft: # of aircraft that the company operates
'''
N = 3
T = 5
H = 10
N_Aircraft = 1200

D = np.array([[[0 for t in range(T+1)] for j in range(N+1)] for i in range(N+1)])
D[1][2][1] = 100
D[1][2][2] = 200
D[1][2][3] = 100
D[1][2][4] = 400
D[1][2][5] = 300
for t in range(1, T+1):
    D[1][3][t] = 50
    D[2][1][t] = 25
    D[2][3][t] = 25
    D[3][1][t] = 40
D[3][2][1] = 400
D[3][2][2] = 200
D[3][2][3] = 300
D[3][2][4] = 200
D[3][2][5] = 400
D[:,:,0] = D[:,:,5]

C = np.array([[0 for j in range(N+1)] for i in range(N+1)])
C[1, 2] = C[2, 1] = 7
C[1, 3] = C[3, 1] = 3
C[2, 3] = C[3, 2] = 6

In [15]:
# Decision Variables
'''
@x_ijt: # of aircraft carrying cargo from location i to j on day t
@y_ijt: # of empty aircraft repositioning from location i to j on day t
@z_it: # of aircraft staying at location i on day t
@r_ijt: units of remaining cargo that should be delivered to j but held at location i on day t 
'''
x_Vars = [[[0 for t in range(T+1)] for j in range(N+1)] for i in range(N+1)]
y_Vars = [[[0 for t in range(T+1)] for j in range(N+1)] for i in range(N+1)]
z_Vars = [[0 for t in range(T+1)] for i in range(N+1)]
r_Vars = [[[0 for t in range(T+1)] for j in range(N+1)]for i in range(N+1)]

myModel = Model("Project")
for i in range(1, N+1):
    for j in range(1, N+1):
        if i != j:
            for t in range(1, T+1):
                curVar = myModel.addVar(vtype = GRB.CONTINUOUS, name = "x_" + str(i)+ "_" + str(j)+ "_" + str(t))
                x_Vars[i][j][t] = curVar
            x_Vars[i][j][0] = x_Vars[i][j][5]

for i in range(1, N+1):
    for j in range(1, N+1):
        if i != j:
            for t in range(1, T+1):
                curVar = myModel.addVar(vtype = GRB.CONTINUOUS, name = "y_" + str(i)+ "_" + str(j)+ "_" + str(t))
                y_Vars[i][j][t] = curVar
            y_Vars[i][j][0] = y_Vars[i][j][5]

for i in range(1, N+1):
        for t in range(1, T+1):
            curVar = myModel.addVar(vtype = GRB.CONTINUOUS, name = "z_" + str(i)+ "_" + str(t))
            z_Vars[i][t] = curVar
        z_Vars[i][0] = z_Vars[i][5]

for i in range(1, N+1):
    for j in range(1, N+1):
        if i != j:
            for t in range(1, T+1):
                curVar = myModel.addVar(vtype = GRB.CONTINUOUS, name = "r_" + str(i)+ "_" + str(j)+ "_" + str(t))
                r_Vars[i][j][t] = curVar
            r_Vars[i][j][0] = r_Vars[i][j][5]
        

myModel.update()

# Add objective functions
objExpr = LinExpr()
for i in range(1, N+1):
    for j in range(1, N+1):
        for t in range(1, T+1):
            objExpr += C[i][j] * y_Vars[i][j][t]        
for i in range(1, N+1):
    for j in range(1, N+1):
        for t in range(1, T+1):
#             if t == 5:
#                 objExpr += 3*H*r_Vars[i][j][t]
#             else:
                objExpr += H*r_Vars[i][j][t]
myModel.setObjective(objExpr, GRB.MINIMIZE)
myModel.update()


# Add aircraft flow balance constraint
for i in range(1, N+1):
    for t in range(1, T+1):
        constExpr = LinExpr()
        constExpr += z_Vars[i][t]
        constExpr -= z_Vars[i][t-1]
        for j in range(1, N+1):
            constExpr += x_Vars[i][j][t] + y_Vars[i][j][t]
            constExpr -= x_Vars[j][i][t-1] + y_Vars[j][i][t-1]
        myModel.addConstr(lhs = constExpr , sense = GRB.EQUAL, rhs = 0 , name = "aircraft_flow_const_"+str(i)+"_"+str(t))

# Add cargo flow balance constraint
for i in range(1, N+1):
    for t in range(1, T+1):
        for j in range(1, N+1):
            constExpr = LinExpr()
            constExpr += r_Vars[i][j][t]
            constExpr -= r_Vars[i][j][t-1]
            constExpr += x_Vars[i][j][t] 
            constExpr -= D[i][j][t] 
            myModel.addConstr(lhs = constExpr , sense = GRB.EQUAL, rhs = 0 , name = "cargo_flow_const_"+str(i)+"_"+str(j)+"_"+str(t))

# Add cargo demand fulfillment constraint
for i in range(1, N+1):
    for j in range(1, N+1):
        constExpr = LinExpr()
        for t in range(1, T+1):
            constExpr += x_Vars[i][j][t]
            constExpr -= D[i][j][t]
        myModel.addConstr(lhs = constExpr , sense = GRB.EQUAL, rhs = 0 , name = "cargo_fulfillment_const_"+str(i)+"_"+str(j))
        
        
# Add aircraft number constraint
for t in range(1, T+1):
    constExpr = LinExpr()
    for i in range(1, N+1):
        constExpr += z_Vars[i][t]
        for j in range(1, N+1):
            constExpr += x_Vars[i][j][t] 
            constExpr += y_Vars[i][j][t]
    myModel.addConstr(lhs = constExpr , sense = GRB.EQUAL, rhs = N_Aircraft , name = "aircraft_number_const_t"+str(t))

myModel.update()
myModel.write("project.lp")

In [16]:
myModel.optimize()

x_Vars, y_Vars, z_Vars, r_Vars

Optimize a model with 74 rows, 105 columns and 345 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Presolve removed 29 rows and 3 columns
Presolve time: 0.01s
Presolved: 45 rows, 102 columns, 252 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.275000e+02   0.000000e+00      0s
      22    1.7925000e+04   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.02 seconds
Optimal objective  1.792500000e+04


([[[0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0]],
  [[0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [<gurobi.Var x_1_2_5 (value 110.0)>,
    <gurobi.Var x_1_2_1 (value 200.0)>,
    <gurobi.Var x_1_2_2 (value 290.0)>,
    <gurobi.Var x_1_2_3 (value 100.0)>,
    <gurobi.Var x_1_2_4 (value 400.0)>,
    <gurobi.Var x_1_2_5 (value 110.0)>],
   [<gurobi.Var x_1_3_5 (value 50.0)>,
    <gurobi.Var x_1_3_1 (value 50.0)>,
    <gurobi.Var x_1_3_2 (value 50.0)>,
    <gurobi.Var x_1_3_3 (value 50.0)>,
    <gurobi.Var x_1_3_4 (value 50.0)>,
    <gurobi.Var x_1_3_5 (value 50.0)>]],
  [[0, 0, 0, 0, 0, 0],
   [<gurobi.Var x_2_1_5 (value 25.0)>,
    <gurobi.Var x_2_1_1 (value 25.0)>,
    <gurobi.Var x_2_1_2 (value 25.0)>,
    <gurobi.Var x_2_1_3 (value 25.0)>,
    <gurobi.Var x_2_1_4 (value 25.0)>,
    <gurobi.Var x_2_1_5 (value 25.0)>],
   [0, 0, 0, 0, 0, 0],
   [<gurobi.Var x_2_3_5 (value 25.0)>,
    <gurobi.Var x_2_3_1 (value 25.0)>,
    <gurobi.Var x_2_3_2 (